 # TDDE16 - Project

### Description
Get song lyrics and detect if the lyrics are containing bad words and then mark the song as “Explicit”. Check from explicit songs in Spotify as golden standard. Replace the explicit word with a suitable non-explicit word.

Using Genius and Spotify API :) 


## Data retrieval

### Spotify - Track Retriever

In [8]:
import requests, json, base64

class TrackRetriever():
    def __init__(self):
        self.request_token()
        
    # Requests an OAuth token required by the Spotify API
    def request_token(self):
        # Client ID and secret for Spotify Application
        client_id = '055a803e06f848a9a6b18ec76aafbe6e'
        client_secret = '756b412f52d54b49bd4e2fa0b2f0df15'
        # Encode client ID and secret in base64
        string = ':'.join([client_id, client_secret])
        encoded = base64.b64encode(string.encode())
        # Add encoding to header
        header = { "Authorization" : "Basic " + encoded.decode('utf-8')}
        data = {"grant_type" : "client_credentials"}
        # Url for requesting OAuth token for Spotify API
        url = "https://accounts.spotify.com/api/token" 
        response = requests.post(url, data=data, headers=header)
        # Store OAuth token in class
        self.token = response.json()['access_token']
        if not self.token:
            print("Error receiving token!")
            print(response.json())
    
    # Gets track id of all songs in a playlist. 
    # Input is an Spotify URI of a playlist
    def get_tracks_from_playlist(self, spotify_uri):
        track_list = []
        split = spotify_uri.split(":")
        user = split[2]
        playlist_id = split[4]
        header = { "Authorization" : "Bearer " + self.token}
        url = "https://api.spotify.com/v1/users/{}/playlists/{}/tracks".format(user, playlist_id)
        response = requests.get(url, headers=header)
        json_res = response.json()
        for track in json_res['items']:
            track_list.append(track['track'])
        return track_list
    
    # Extracts track title, artist name and explicit bool from a track json
    def extract_info(self, track):
        # Join all artists to one string separated by a whitespace
        artists = " ".join([str(art['name']) for art in track['artists']])
        # Extract title and explicit status
        title = track['name']
        explicit = track['explicit']
        track_id = track['id']
        return {'id': track_id, 'artist': artists, 'title': title, 'explicit': explicit}
    
    # Starting method for retrieving songs from a playlist
    def get_track_info(self, uri):
        track_list = []
        tracks = self.get_tracks_from_playlist(uri)
        for t in tracks:
            track_list.append(self.extract_info(t))
        return track_list
            
        
# spotify:user:spotify:playlist:37i9dQZF1E9RVkbMAXdy3v        

In [ ]:
tr = TrackRetriever()
tr.get_track_info("spotify:user:spotify:playlist:37i9dQZF1E9RVkbMAXdy3v")

### Genius - Lyrics Catcher

In [19]:
import requests, json, re
from bs4 import BeautifulSoup

class LyricsCatcher():
    def __init__(self, token):
        self.oauth_token = token
        self.api_url = "http://api.genius.com"
        self.api_headers = {'Authorization': 'Bearer ' + oauth_token}
        self.web_url = "https://genius.com"
        
    def search_song(self, song_title, artist_name):
        search_url = "{}/search?q={}{}".format(self.api_url, song_title, artist_name)
        response = requests.get(search_url, headers=self.api_headers)
        json = response.json()
        song_info = None
        for hit in json["response"]["hits"]:
            if hit["result"]["primary_artist"]["name"] == artist_name:
                song_info = hit
                break
        return song_info
    
    def get_url(self, song_id):
        endpoint = "/songs/{}".format(song_id)
        # Get song info from API using the song ID
        response = requests.get(self.api_url + endpoint, headers=self.api_headers)
        return response.json()["response"]["song"]["path"]
    
    def get_lyrics(self, endpoint):
        # Fetch page containing the lyrics
        u = self.web_url + endpoint
        page = requests.get(u)
        # Extract HTML source code from page
        html = BeautifulSoup(page.text, "html.parser")
        # Remove script tags that they put in the middle of the lyrics
        [h.extract() for h in html('script')]
        # Get the div-tag where the lyrics are and extract text inside
        lyrics = html.find("div", class_="lyrics").get_text()
        lyrics = re.sub(r'\[.+\]', '', lyrics)
        lyrics = re.sub(r'\n+', ' ', lyrics)
        return lyrics
    
    def fetch_lyrics(self, title, artist):
        song = self.search_song(title, artist)
        if not(song):
            return None
        song_id = str(song["result"]["id"])
        path = self.get_url(song_id)
        lyrics = self.get_lyrics(path)
        return lyrics
        

## Data gathering
Using the Spotify API to get songs and explicit status and then use the genius API to search for lyrics

In [20]:
# Method for printing statistics of the tracks
def evaluate(tracks):
    return ""

In [18]:
oauth_token = "O8rraozwWRK1i66ofhUJSZ9EDCBLkxg-gfoDzBB9_XH3Vjsm0qYoewK8C7lMOWkh"
catcher = LyricsCatcher(oauth_token)
tr = TrackRetriever()
playlists = [
    'spotify:user:heekzz:playlist:5IeIXRXPSkRpBX9bNo5onK', # Test
    #'spotify:user:heekzz:playlist:6nL0gtJLo2xqCWIirZ4inF', # Rockify
    #'spotify:user:spotify:playlist:37i9dQZF1DXcF6B6QPhFDv', # Rock Classics
    #'spotify:user:spotify:playlist:37i9dQZF1DX0XUsuxWHRQd', # Rapcaviar 
    #'spotify:user:spotify:playlist:37i9dQZF1DWVA1Gq4XHa6U', # Gold School (Hiphop)
    #'',
    #'',
    #'',
    #'',
    #'',
    #'',
    #''
]

tracks_with_lyrics = []
track_id_set = set()
for pl in playlists:
    tracks = tr.get_track_info(pl)
    for t in tracks:
        if t['id'] not in track_id_set:
            lyrics = catcher.fetch_lyrics(t['title'], t['artist'])
            if lyrics != None:
                data = {'id': t['id'], 'artist': t['artist'], 'title': t['title'], 'explicit': t['explicit'], 'lyrics': lyrics}
                tracks_with_lyrics.append(data)
            track_id_set.add(t['id'])
print(tracks_with_lyrics)

[{'id': '3fsghmcEnT7K9f30ohHDRn', 'artist': '50 Cent', 'title': 'In Da Club', 'explicit': True, 'lyrics': ' Go, go, go, go, go, go Go shorty, it\'s your birthday We gonna party like it\'s your birthday We gonna sip Bacardi like it\'s your birthday And you know we don\'t give a fuck it\'s not your birthday You can find me in the club, bottle full of bub\' Look, mami, I got the X if you into takin\' drugs I\'m into havin\' sex, I ain\'t into makin\' love So come give me a hug if you\'re into gettin\' rubbed You can find me in the club, bottle full of bub\' Look, mami, I got the X if you into takin\' drugs I\'m into havin\' sex, I ain\'t into makin\' love So come give me a hug if you\'re into gettin\' rubbed When I pull up out front, you see the Benz on dubs When I roll twenty deep it\'s twenty knives in the club Niggas heard I fuck with Dre, now they wanna show me love When you sell like Eminem and the hoes, they wanna fuck But homie, ain\'t nothin\' changed: hoes down, G\'s up I see Xzi